In [ ]:
import pandas as pd
import matplotlib as plt

Import datasets

In [ ]:
d13 = pd.read_stata('./data/2013', convert_categoricals=False)
d14 = pd.read_stata('./data/2014', convert_categoricals=False)
d15 = pd.read_stata('./data/2015', convert_categoricals=False)
d16 = pd.read_stata('./data/2016', convert_categoricals=False)
d17 = pd.read_stata('./data/2017', convert_categoricals=False)
d18 = pd.read_stata('./data/2018', convert_categoricals=False)
d19 = pd.read_stata('./data/2019', convert_categoricals=False)
d20 = pd.read_stata('./data/2020', convert_categoricals=False)
d21 = pd.read_stata('./data/2021', convert_categoricals=False)
d22 = pd.read_stata('./data/2022', convert_categoricals=False)

Recode pre-2017 LFSSTAT variables

In [ ]:
pre = pd.concat([d13, d14, d15, d16])

def recode(LFSSTAT):
	if LFSSTAT == 4:
		return 3
	elif LFSSTAT == 5:
		return 3
	elif LFSSTAT == 6:
		return 4
	else:
		return LFSSTAT

pre['LFSSTAT'] = pre['LFSSTAT'].apply(recode)

Append the rest of the data to the master

In [ ]:
post = pd.concat([d17, d18, d19, d20, d21, d22])
master = pd.concat([pre, post])